In [4]:
#Chatbot using NLP

In [3]:
# Importing necessary libraries
import nltk
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


In [5]:
ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mansi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
intents = [
    {
        'tag': 'greeting',
        'patterns': ['Hi', 'Hello', 'Hey', 'Whats up', 'How are you', 'Good morning', 'Good evening'],
        'responses': ['Hi there!', 'Hello, how can I assist you?', 'Hey, how’s it going?', 'Good morning! How can I help?', 'Good evening, how are you doing?']
    },
    {
        'tag': 'goodbye',
        'patterns': ['Bye', 'See you later', 'Goodbye', 'Take care', 'Catch you later'],
        'responses': ['Goodbye!', 'See you next time!', 'Take care!', 'Catch you later!', 'Have a great day!']
    },
    {
        'tag': 'thanks',
        'patterns': ['Thank you', 'Thanks', 'Thanks a lot', 'I appreciate it', 'Thanks so much'],
        'responses': ['You\'re welcome!', 'No worries!', 'Glad to help!', 'Anytime!', 'You got it!']
    },
    {
        'tag': 'about',
        'patterns': ['What can you do', 'Who are you', 'What are you', 'What is your purpose', 'Tell me about yourself'],
        'responses': ['I am your friendly chatbot assistant!', 'My purpose is to help with anything you need.', 'I assist with a wide range of tasks and questions.', 'I’m here to make your life easier!']
    },
    {
        'tag': 'help',
        'patterns': ['Help', 'I need help', 'Can you help me', 'What should I do', 'I’m stuck'],
        'responses': ['Sure, what do you need help with?', 'I’m here to help. What’s the problem?', 'How can I assist you today?', 'Let me know how I can help you!']
    },
    {
        'tag': 'age',
        'patterns': ['How old are you', 'What\'s your age', 'When were you created', 'Are you old?'],
        'responses': ['I don’t have an age like humans do. I live in the digital world.', 'I was created in the digital realm, so no age for me!', 'Age is just a number for me.']
    },
    {
        'tag': 'weather',
        'patterns': ['What\'s the weather like', 'How\'s the weather today', 'Is it raining', 'Will it snow today'],
        'responses': ['I’m sorry, I can’t check the weather for you.', 'Check a weather app for the most accurate information!', 'I can’t access live weather updates, but a quick search will help!']
    },
    {
        'tag': 'budget',
        'patterns': ['How can I make a budget', 'What\'s a good budgeting strategy', 'How do I create a budget', 'Can you help me manage my money'],
        'responses': ['Start by tracking your income and expenses. Set aside funds for necessities, savings, and some fun too!', 'The 50/30/20 rule is a great strategy for budgeting. 50% for essentials, 30% for discretionary, and 20% for savings/debt.', 'Identify your financial goals, track your spending, and allocate income for essentials and savings.']
    },
    {
        'tag': 'credit_score',
        'patterns': ['What is a credit score', 'How do I check my credit score', 'How can I improve my credit score', 'What affects my credit score'],
        'responses': ['A credit score is a number that represents your creditworthiness based on your financial history.', 'You can check your credit score on free sites like Credit Karma or through your bank.', 'To improve your credit score, focus on paying bills on time, reducing debt, and checking your credit report for errors.']
    },
    {
        'tag': 'fitness',
        'patterns': ['How do I get fit', 'What should I eat to be healthier', 'How can I lose weight', 'Can you suggest a workout'],
        'responses': ['Start with regular exercise and balanced meals. Aim for a mix of cardio, strength training, and flexibility exercises.', 'For a healthy lifestyle, focus on whole foods, regular exercise, and staying hydrated.', 'A good workout routine includes cardio, strength exercises, and rest days. Start slow and gradually increase intensity.']
    },
    {
        'tag': 'motivation',
        'patterns': ['I need motivation', 'I\'m feeling unmotivated', 'Can you inspire me', 'Give me some advice'],
        'responses': ['You’ve got this! Remember, small steps lead to big changes.', 'Don’t give up! Every day is a new chance to start fresh.', 'Believe in yourself. Progress is progress, no matter how small!']
    },
    {
        'tag': 'time',
        'patterns': ['What time is it', 'Can you tell me the time', 'What’s the current time', 'What’s the time now'],
        'responses': ['I’m unable to provide real-time data, but you can check your device for the current time!', 'You can always check the time on your phone or watch!']
    },
    {
        'tag': 'vacation',
        'patterns': ['Where should I go on vacation', 'Can you suggest a travel destination', 'What’s a good place to visit', 'I want to travel, where to go'],
        'responses': ['How about a tropical beach or a scenic mountain retreat? Let me know what you like!', 'There are many great vacation spots depending on your interests. Do you like beaches, cities, or nature?', 'A visit to a national park or a cultural city might be great. What type of vacation are you looking for?']
    },
    {
        'tag': 'learning',
        'patterns': ['How can I learn something new', 'What’s the best way to learn', 'I want to learn something new', 'Can you teach me'],
        'responses': ['Start with setting clear goals and breaking the learning process into small steps.', 'There are many ways to learn: books, online courses, and hands-on experience. Find what works for you!', 'The key to learning is consistency. Stay curious and keep practicing!']
    },
    {
        'tag': 'relationship',
        'patterns': ['Can you give me relationship advice', 'What do I do if I’m having relationship problems', 'How can I improve my relationship'],
        'responses': ['Communication is key! Make sure to talk openly with your partner about your feelings.', 'Relationships require trust, respect, and empathy. Listen and understand each other’s needs.', 'Remember to make time for each other and prioritize your bond. Relationships take work, but they’re worth it!']
    },
    {
        'tag': 'sleep',
        'patterns': ['How do I sleep better', 'What can I do if I can’t sleep', 'I have trouble sleeping', 'Can you help me sleep'],
        'responses': ['Try a bedtime routine: limit screen time, stay relaxed, and create a comfortable environment.', 'A calming environment, like soft lighting and a quiet room, helps. Also, try not to drink caffeine too late.', 'Sleep hygiene is important. Consistency in your sleep schedule, limiting naps, and winding down before bed are helpful!']
    },
    {
        'tag': 'productivity',
        'patterns': ['How do I stay productive', 'I can’t focus on my work', 'How do I get things done', 'Can you help me with productivity'],
        'responses': ['Set clear goals, break tasks into smaller chunks, and take regular breaks. Start with the hardest task first!', 'Eliminate distractions, prioritize your tasks, and use productivity techniques like Pomodoro.', 'Stay organized! Use tools like to-do lists, and track your progress. Small, consistent efforts lead to big results!']
    }
]
